To begin, start iPython interface in Inline Pylab mode by typing following on your terminal / windows command prompt:
In [1]:


In [ ]:
%pylab inline


After importing the library, you read the dataset using function read_csv(). The file is assumed to be downloaded from the moodle to the data folder in your working directory.

In [ ]:
plot(arange(5))

Following are the libraries we will use during this task:
numpy
matplotlib
pandas
Please note that you do not need to import matplotlib and numpy because of Pylab environment. I have still kept them in the code, in case you use the code in a different environment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as plt
from collections import Counter
from numpy.random import randn
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score,accuracy_score

rcParams['figure.figsize'] = 16, 6


After importing the library, you read the dataset using function read_csv(). The file is assumed to be downloaded from the moodle to the data folder in your working directory.

In [ ]:
train = pd.read_csv("C:/Users/User/Desktop/ML/EX4-data/train.csv") #Reading the dataset in a dataframe using Pandas
test = pd.read_csv("C:/Users/User/Desktop/ML/EX4-data/test.csv") #Reading the dataset in a datafra

Now that we are familiar with basic data characteristics, let us study distribution of various variables. Let us start with numeric variables – namely ApplicantIncome and LoanAmount

Lets start by plotting the histogram of ApplicantIncome using the following commands:

In [ ]:
train['ApplicantIncome'].hist(bins=50)

Here we observe that there are few extreme values. This is also the reason why 50 bins are required to depict the distribution clearly.
Next, we look at box plots to understand the distributions. Box plot can be plotted by:

In [ ]:
train.boxplot(column='ApplicantIncome')

This confirms the presence of a lot of outliers/extreme values. This can be attributed to the income disparity in the society. Part of this can be driven by the fact that we are looking at people with different education levels. Let us segregate them by Education:

In [ ]:
train.boxplot(column='ApplicantIncome', by = 'Education')

We can see that there is no substantial different between the mean income of graduate and non-graduates. But there are a higher number of graduates with very high incomes, which are appearing to be the outliers.

Plot the histogram and boxplot of LoanAmount


In [ ]:
train['LoanAmount'].hist(bins=50)

In [ ]:
train.boxplot(column='LoanAmount')

Again, there are some extreme values. Clearly, both ApplicantIncome and LoanAmount require some amount of data munging. LoanAmount has missing and well as extreme values values, while ApplicantIncome has a few extreme values, which demand deeper understanding. We will take this up in coming sections.


Frequency Table for Credit History:


In [ ]:
temp1 = train['Credit_History'].value_counts(ascending=True)
temp1

Probability of getting loan for each Credit History class:

In [ ]:
temp2 = train.pivot_table(values='Loan_Status',index=['Credit_History'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
temp2

This can be plotted as a bar chart using the “matplotlib” library with following code:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Credit_History')
ax1.set_ylabel('Count of Applicants')
ax1.set_title("Applicants by Credit_History")
temp1.plot(kind='bar')

ax2 = fig.add_subplot(122)
temp2.plot(kind = 'bar')
ax2.set_xlabel('Credit_History')
ax2.set_ylabel('Probability of getting loan')
ax2.set_title("Probability of getting loan by credit h


This shows that the chances of getting a loan are eight-fold if the applicant has a valid credit history. You can plot similar graphs by Married, Self-Employed, Property_Area, etc.
Alternately, these two plots can also be visualized by combining them in a stacked chart::

In [ ]:
temp3 = pd.crosstab(train['Credit_History'], train['Loan_Status'])
temp3.plot(kind='bar', stacked=True, color=['red','blue'], grid=False)

We just saw how we can do exploratory analysis in Python using Pandas. I hope your love for pandas (the animal) would have increased by now – given the amount of help, the library can provide you in analyzing datasets.
Next let’s explore ApplicantIncome and LoanStatus variables further, perform data munging and create a dataset for applying various modeling techniques. I would strongly urge that you take another dataset and problem and go through an independent example before reading further.

While our exploration of the data, we found a few problems in the data set, which needs to be solved before the data is ready for a good model. This exercise is typically referred as “Data Munging”. Here are the problems, we are already aware of:
1. There are missing values in some variables. We should estimate those values wisely depending on the amount of missing values and the expected importance of variables.
2. While looking at the distributions, we saw that ApplicantIncome and LoanAmount seemed to contain extreme values at either end. Though they might make intuitive sense, but should be treated appropriately.

In addition to these problems with numerical fields, we should also look at the non-numerical fields i.e. Gender, Property_Area, Married, Education and Dependents to see, if they contain any useful information.

Let us look at missing values in all the variables because most of the models don’t work with missing data and even if they do, imputing them helps more often than not. So, let us check the number of nulls / NaNs in the dataset.
This command should tell us the number of missing values in each column as isnull() returns 1, if the value is null.

In [ ]:
train.apply(lambda x: sum(x.isnull()),axis=0)

Though the missing values are not very high in number, but many variables have them and each one of these should be estimated and added in the data.

Note: Remember that missing values may not always be NaNs. For instance, if the Loan_Amount_Term is 0, does it makes sense or would you consider that missing? I suppose your answer is missing and you’re right. So we should check for values which are unpractical.

How to fill missing values in LoanAmount?
There are numerous ways to fill the missing values of loan amount – the simplest being replacement by mean, which can be done by following code:

In [ ]:
train['Self_Employed'].value_counts()


The other extreme could be to build a supervised learning model to predict loan amount on the basis of other variables and then use age along with other variables to predict survival.

Since, the purpose now is to bring out the steps in data munging, I’ll rather take an approach, which lies some where in between these 2 extremes. A key hypothesis is that whether a person is educated or self-employed can combine to give a good estimate of loan amount.

But first, we have to ensure that each of Self_Employed and Education variables should not have a missing values.

As we say earlier, Self_Employed has some missing values. Let’s look at the frequency table:

In [ ]:
train['Self_Employed'].value_counts()

Since ~86% values are “No”, it is safe to impute the missing values as “No” as there is a high probability of success. This can be done using the following code:

In [ ]:
train['Self_Employed'].fillna('No',inplace=True)

Now, we will create a Pivot table, which provides us median values for all the groups of unique values of Self_Employed and Education features. Next, we define a function, which returns the values of these cells and apply it to fill the missing values of loan amount:

In [ ]:
table = train.pivot_table(values='LoanAmount', index='Self_Employed' ,columns='Education', aggfunc=np.median)
table

Define function to return value of this pivot_table:


In [ ]:
def fage(x):
 return table.loc[x['Self_Employed'],x['Education']]

Replace missing values:


In [ ]:
train['LoanAmount'].fillna(train[train['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)

This should provide you a good way to impute missing values of loan amount.


Let’s analyze LoanAmount first. Since the extreme values are practically possible, i.e. some people might apply for high value loans due to specific needs. So instead of treating them as outliers, let’s try a log transformation to nullify their effect:

In [ ]:
train['LoanAmount_log'] = np.log(train['LoanAmount'])
train['LoanAmount_log'].hist(bins=20)

Now the distribution looks much closer to normal and effect of extreme values has been significantly subsided.
Coming to ApplicantIncome. One intuition can be that some applicants have lower income but strong support Co-applicants. So it might be a good idea to combine both incomes as total income and take a log transformation of the same.

In [ ]:
train['TotalIncome'] = train['ApplicantIncome'] + train['CoapplicantIncome']
train['TotalIncome_log'] = np.log(train['TotalIncome'])
train['LoanAmount_log'].hist(bins=20)

Now we see that the distribution is much better than before.

1. Impute the missing values for Gender, Married, Dependents, Loan_Amount_Term, Credit_History
2. Also, I encourage you to think about possible additional information which can be derived from the data. For example, creating a column for LoanAmount/TotalIncome might make sense as it gives an idea of how well the applicant is suited to pay back his loan.

In [ ]:
train['Loan_Amount_Term'].fillna(360, inplace=True)
train['Credit_History'].fillna(1, inplace=True)

In [ ]:
train['Dependents'].value_counts()

In [ ]:
train['Dependents'].fillna(0, inplace=True)

In [ ]:
train['Married'].value_counts()

In [ ]:
train['Married'].fillna('Yes', inplace=True)

In [ ]:
train['Gender'].value_counts()

In [ ]:
train['Married'].fillna('Male', inplace=True)

Next, we will look at making predictive models.

After, we have made the data useful for modeling, let’s now look at the python code to create a predictive model on our data set. Skicit-Learn (sklearn) is the most commonly used library in Python for this purpose and we will follow the trail.

Since, sklearn requires all inputs to be numeric, we should convert all our categorical variables into numeric by encoding the categories. This can be done using the following code:

In [ ]:
train.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i].astype(str))

In [ ]:
train.dtypes

Next, we will import the required modules. Then we will define a generic classification function, which takes a model as input and determines the Accuracy and Cross-Validation scores.

In [ ]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(data.shape[0], n_folds=5)
  error = []
  for train, test in kf:
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome])

Let’s make our first Logistic Regression model. One way would be to take all the variables into the model but this might result in overfitting. In simple words, taking all variables might result in the model understanding complex relations specific to the data and will not generalize well.
We can easily make some intuitive hypothesis to set the ball rolling. The chances of getting a loan will be higher for:
1. Applicants having a credit history (remember we observed this in exploration?)
2. Applicants with higher applicant and co-applicant incomes
3. Applicants with higher education level
4. Properties in urban areas with high growth perspectives
So let’s make our first model with ‘Credit_History’.

In [ ]:
outcome_var = 'Loan_Status'
model = LogisticRegression()
predictor_var = ['Credit_History']
classification_model(model, train,predictor_var,outcome_var)

We can try different combination of variables:

In [ ]:
predictor_var = ['Credit_History','Education','Married','Self_Employed','Property_Area']
classification_model(model, train,predictor_var,outcome_var)

Generally we expect the accuracy to increase on adding variables. But this is a more challengin case. The accuracy and cross-validation score are not getting impacted by less important variables. Credit_History is dominating the mode. We have two options now:
1. Feature Engineering: dereive new information and try to predict those. I will leave this to your creativity.
2. Better modeling techniques. Let’s explore this next.

 Decision tree is another method for making a predictive model. It is known to provide higher accuracy than logistic regression model.
In [35]:



In [ ]:
model = DecisionTreeClassifier()
predictor_var = ['Credit_History','Gender','Married','Education']
classification_model(model, train,predictor_var,outcome_var)

Here the model based on categorical variables is unable to have an impact because Credit History is dominating over them. Let’s try a few numerical variables:

1. Train a decision tree classifier that accepts the following attributes as input: Credit_History, Loan_Amount_Term, LoanAmount_log
2. Is it better that the previous decision tree?

In [ ]:
# model = DecisionTreeClassifier()
predictor_var = ['Credit_History','Loan_Amount_Term', 'LoanAmount_log']
classification_model(model, train, predictor_var, outcome_var)

Accuracy is significantly better, but cross-validation is worse. In overall performance it might be worst than the prevoius one, as it seems like an overfitting issue.

In [ ]:
#We can try different combination of variables:
predictor_var = ['Credit_History','Loan_Amount_Term','LoanAmount_log']
classification_model(model, train,predictor_var,outcome_var)

2.Not much better according to the test score
Here we observed that although the accuracy went up on adding variables, the cross-validation error went down. This is the result of model over-fitting the data. Let’s try an even more sophisticated algorithm and see if it helps:


Random forest is another algorithm for solving the classification problem.
An advantage with Random Forest is that we can make it work with all the features and it returns a feature importance matrix which can be used to select features.

In [ ]:
model = RandomForestClassifier(n_estimators=100)
predictor_var = ['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
        'LoanAmount_log','TotalIncome_log']
classification_model(model, train,predictor_var,outcome_var)

Accuracy : 100.000%
Cross-Validation Score : 77.856%
Here we see that the accuracy is 100% for the training set. This is the ultimate case of overfitting and can be resolved in two ways:
Reducing the number of predictors
Tuning the model parameters
Let’s try both of these. First we see the feature importance matrix from which we’ll take the most important features.

In [ ]:
#Create a series with feature importances:
featimp = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(featimp)

Let’s use the top 5 variables for creating a model. Also, we will modify the parameters of random forest model a little bit:

Let’s use the top 5 variables for creating a model. Also, we will modify the parameters of random forest model a little bit:

In [ ]:
model = RandomForestClassifier(n_estimators=25, min_samples_split=25, max_depth=7, max_features=1)
predictor_var = ['TotalIncome_log','LoanAmount_log','Credit_History','Dependents','Property_Area']
classification_model(model, train,predictor_var,outcome_var)

Now, we start.
We saw erlier the columns with missing values:
Gender 13
Married 3
Depentents 15
Self_Employed 32
LoanAmount 22
Loan_Amount_Term 14
Credit_History 50

lets check what is going with the train set.

We will divide the train set into two,one with the wanted column and one without.


In [ ]:
train = pd.read_csv("C:/Users/moran beladev/Desktop/Studies/IDF/data/train.csv")
test = pd.read_csv("C:/Users/moran beladev/Desktop/Studies/IDF/data/test.csv")

trainX = train.drop('Loan_Status', axis=1, inplace=False)
trainX = train.drop('Loan_Status', axis=1, inplace=False)

testX = test

Let's examine the values in these ocolumns: Married, depentdents and Gender since it is more easy to fill those.

In [ ]:
print(trainX['Married'].value_counts(),'\n')
print(trainX['Dependents'].value_counts(),'\n')
print(trainX['Gender'].value_counts(),'\n')

In [ ]:
Now, we will do it on the test too

In [ ]:
print(testX['Married'].value_counts(),'\n')
print(testX['Dependents'].value_counts(),'\n')
print(testX['Gender'].value_counts(),'\n'

We fill in the missing value with the most accured one 

In [ ]:
trainX['Dependents'].fillna(0, inplace=True)
trainX['Married'].fillna('Yes', inplace=True)
trainX['Gender'].fillna('Male', inplace=True)

now lets take care of the rest missing values of 'Loan_amount_term', 'loanAmount'and
'Credit_History' features

In [ ]:
trainX['LoanAmount'].fillna(trainX.LoanAmount.dropna().mode().values[0], inplace=True)
trainX['Loan_Amount_Term'].fillna(trainX.Loan_Amount_Term.dropna().mode().values[0], inplace=True)
trainX['Self_Employed'].fillna(trainX.Self_Employed.dropna().mode().values[0], inplace=True)

the same for testX set


In [ ]:
testX['Dependents'].fillna(0, inplace=True)
testX['Married'].fillna('Yes', inplace=True)
testX['Gender'].fillna('Male', inplace=True)
#please note we fill in the missing values based on the training set
testX['LoanAmount'].fillna(trainX.LoanAmount.dropna().mode().values[0], inplace=True)
testX['Loan_Amount_Term'].fillna(trainX.Loan_Amount_Term.dropna().mode().values[0], inplace=True)
testX['Self_Employed'].fillna(trainX.Self_Employed.dropna().mode().values[0], inplace=True)

now there is only the 'Credit_history' feature we still need to handle


In [ ]:
trainX['Credit_History'].value_counts()

for this feature we will train the set, let's combine the two sets into one

In [ ]:
from sklearn.preprocessing import LabelEncoder
var_mod = {'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'}
le = LabelEncoder()
for i in var_mod:
    trainX[i] = le.fit_transform(trainX[i].astype(str))
    testX[i] = le.fit_transform(testX[i].astype(str))
trainY=le.fit_transform(trainY.astype(str))
tmp_trainX = trainX[trainX.Credit_History.notnull()]
randomForest = RandomForestClassifier(random_state=np.random)
randomForest.fit(tmp_trainX.drop(['Credit_History', 'Loan_ID'], axis=1), tmp_trainX['Credit_History'])

testX_i = testX[testX.Credit_History.isnull()]
testY_Predict = randomForest.predict(testX_i.drop(['Credit_History', 'Loan_ID','Loan_Status'], axis=1))
testX.loc[testX.Credit_History.isnull(),'Credit_History'] = testY_Predict

trainX_i = trainX[trainX.Credit_History.isnull()]
trainY_Predict = randomForest.predict(trainX_i.drop(['Credit_History', 'Loan_ID'], axis=1))
trainX.loc[trainX.Credit_History.isnull(),'Credit_History'] = trainY_Predict

We will use KNN Classifier - K nearest neighbors algorithm
K Nearest Neighbors  Classification is a simple algorithm that stores all available cases and classifies new cases
based on a similarity measure (e.g., distance functions). KNN has been used in statistical estimation and pattern
recognition already in the beginning of 1970’s as a non-parametric technique


In [ ]:
# KNNClassifier = KNeighborsClassifier()
KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', p=2)
KNN.fit(trainX.drop(['Loan_ID'],axis=1), trainY)

loan_ids=testX['Loan_ID']
result = pd.DataFrame({'Loan_ID': loan_ids.values, 'Loan_Status': KNN.predict(testX.drop(['Loan_ID','Loan_Status'],axis=1))})
result.to_csv('submission1.csv', index=False)

 Now we will use Random Forest Classifier -

In [ ]:
abc = AdaboostClassifier (n_estimators=50, learning_rate=0.5, random_state = 11 )
abc.fit()trainX.drop(['Loan_ID'],axis=1), trainY)

loan_ids=testX['Loan_ID']
result = pd.DataFrame({'Loan_ID': loan_ids.values, 'Loan_Status': abc.predict(testX.drop(['Loan_ID','Loan_Status'],axis=1))})
result.to_csv('submission2.csv', index=False)